# Advanced McStas features: EXTEND and WHEN
In this tutorial we will look at two advanced features in McStas, the EXTEND block and WHEN condition. Here we will use them to flag certain neutrons with EXTEND, and only record them in monitors when the flag is set using a WHEN condition.

In [ ]:
import mcstasscript as ms

In [ ]:
instrument = ms.McStas_instr("python_tutorial", input_path="run_folder",
                                output_path="data_folder/mcstas_EXTEND_WHEN")

### Set up an example McStas instrument
First we set up an example instrument conisiting of a source, a guide and a position/divergence monitor. The guide is set up such that it only has mirrors on the left and right side, and absorbs neutrons if they hit the top or bottom. This is done to look at the horizontal behavior independently from the vertical, as this is easier to analyze.

In [ ]:
src = instrument.add_component("source", "Source_simple")

src.xwidth = 0.02
src.yheight = 0.02
src.focus_xw = guide_opening_w = 0.05
src.focus_yh = guide_opening_h = 0.06
src.dist = 1.5
src.flux = 1E13

wavelength = instrument.add_parameter("wavelength", value=5.0,
                                      comment="Wavelength in [Ang]")
src.lambda0 = wavelength
src.dlambda = "0.001*wavelength"

In [ ]:
guide = instrument.add_component("guide", "Guide_gravity", AT=[0,0,1.5], RELATIVE=src)
guide.w1 = guide_opening_w
guide.h1 = guide_opening_h
guide.w2 = guide_opening_w
guide.h2 = guide_opening_h
guide.l = guide_length = 15
guide.mleft = 4.0
guide.mright = 4.0
guide.mtop = 0.0
guide.mbottom = 0.0
guide.G = -9.82

In [ ]:
acceptance = instrument.add_component("acceptance", "DivPos_monitor")
acceptance.set_AT([0,0, guide_length + 0.1], RELATIVE=guide)
acceptance.nh = 200
acceptance.ndiv = 200
acceptance.filename = '"acceptance.dat"'
acceptance.xwidth = 0.08
acceptance.yheight = 0.05
acceptance.maxdiv_h = 1.5
acceptance.restore_neutron = 1

In [ ]:
instrument.set_parameters(wavelength=2.8)
instrument.settings(ncount=5E6)
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data)

### Interpreting the data
Here we see an acceptance monitor, with position along the x-axis and divergence along the y-axis. The guide is under illuminated by the small source, so there are gaps in the acceptance diagram. We see the position and divergence of the beam consist of a large number of stripes, the ones with lowest divergence has the largest intensity.

## Add an flag
A flag is just a name for a variable that records some information on the neutron during the simulation, and can be used later to make a decision. Here we could check how many times the ray was reflected in the guide.

We use an EXTEND block after a component to access variables internal to the component in the instrument scope. We declare a variable in the instrument scope called *n_reflections*. In the component scope, one can use the SCATTERED variable which contains the number of times the ray has encountered the SCATTER keyword within the component. Usually this is done when entering and leaving, and under each scattering / reflection, so the number of reflections is SCATTERED - 2.

In [ ]:
instrument.add_declare_var("int", "n_reflections")
guide.append_EXTEND("n_reflections = SCATTERED - 2;")
guide.print_long()

## Use the flag to limit what is recorded in a monitor
A WHEN statement can be used to activate / deactivate a component when some condition is true / false. For example we could require 0 reflection in our guide. We add a few monitors similar to the original, with the only difference being WHEN statements requiring 0, 1 or 2 reflections in the guide for the component to be active. We use a for loop to add the similar components, only changing the component instance name, filename and WHEN statement between each.

In [ ]:
reflection_numbers = [0, 1, 2]

for reflections in reflection_numbers:
    reflections_string = str(reflections)
    
    component_name = "acceptance_" + reflections_string
    acceptance = instrument.add_component(component_name, "DivPos_monitor")
    acceptance.filename = '"acceptance_' + reflections_string + '.dat"'
    acceptance.set_WHEN("n_reflections == " + reflections_string)
    
    acceptance.set_AT([0,0, guide_length + 0.1], RELATIVE=guide)
    acceptance.nh = 200
    acceptance.ndiv = 200
    acceptance.xwidth = 0.08
    acceptance.yheight = 0.05
    acceptance.maxdiv_h = 1.5
    acceptance.restore_neutron = 1
    
    acceptance.print_long()
    print("")

### Running the simulation
We now run the simulation with the new monitors to see how they differ from the original version.

In [ ]:
instrument.set_parameters(wavelength=2.8)
instrument.settings(ncount=5E6)
instrument.show_settings()

new_data = instrument.backengine()

In [ ]:
ms.make_sub_plot(new_data)

### Interpretation of the data
The original monitor is unchanged as it was not modified. On the monitors with different numbers of reflections, we see the middle line correspond to zero reflections, the two lines around those are for one reflection and so forth. This explains why the lines further from the center has lower intensity, as they underwent more reflections while also having a larger angle of incidence.

### The McStas instrument file
We here show the generated McStas instrument file in order to clarify how this would be accomplished without the McStasScript API.

In [ ]:
with open("run_folder/python_tutorial.instr") as file:
    instrument_string = file.read()
    print(instrument_string)